In [59]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
df = pd.read_csv("Features_all.csv")
dfs = df[(df.modeID != 'audible' )]
df

,channel_1_mav,channel_1_rms,channel_1_iav,channel_1_var,channel_1_ssi,channel_1_var.1,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_medf,channel_6_meanf,userID,sessionID,uttID,modeID
0,0.303753,0.428991,1186.761932,0.179216,719.016443,0.179216,1741.765442,1186.761932,0.445806,1260,...,0.444535,1278,2241,1068,125.706941,124.086980,1,1001,10010001,audible
1,0.302680,0.428737,795.746155,0.179036,483.250393,0.179036,1175.243469,795.746155,0.447031,827,...,0.443308,794,1497,728,112.643678,120.463170,1,1001,10010002,audible
2,0.303781,0.429230,1437.794739,0.179372,872.001527,0.179372,2109.147064,1437.794739,0.445626,1528,...,0.445901,1534,2726,1316,126.483051,128.359225,1,1001,10010003,audible
3,0.304343,0.429568,1478.802124,0.179585,896.626874,0.179585,2167.499481,1478.802124,0.446079,1538,...,0.445519,1540,2792,1345,127.685950,127.765869,1,1001,10010004,audible
4,0.305522,0.430619,1675.482758,0.180567,1016.913968,0.180567,2448.796021,1675.482758,0.446535,1705,...,0.446329,1716,3143,1523,137.111517,132.848277,1,1001,10010005,audible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7374,0.303448,0.429408,660.909515,0.179325,401.604101,0.179325,971.741547,660.909515,0.446162,690,...,0.445487,706,1252,590,115.277778,119.935476,8,8101,81010506,audible
7375,0.302741,0.428617,689.644287,0.178358,418.497516,0.178358,1013.215820,689.644287,0.444783,727,...,0.443333,702,1307,627,106.194690,115.396655,8,8101,81010507,audible
7376,0.303156,0.428916,599.339050,0.178800,363.706382,0.178800,881.657990,599.339050,0.445958,637,...,0.443360,624,1141,540,140.816327,130.759108,8,8101,81010508,audible
7377,0.305219,0.430511,695.595184,0.180370,422.389347,0.180370,1020.579163,695.595184,0.447819,733,...,0.444682,722,1317,622,112.831858,119.083347,8,8101,81010509,audible


In [60]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['userID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data():
    temp_df = df
    data = temp_df.copy()
    data = shuffle(data)
    y = data['userID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_per_user(user_id):
    temp_df = df[(df.userID == user_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['userID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

In [61]:
def lda_all_users_separately():
    user_ids = np.unique(df.userID)
    for user_id in user_ids:
        X_train, X_test, y_train, y_test = split_data_per_user(user_id)
        k_clf = LinearDiscriminantAnalysis()
        k_clf.fit(X_train, y_train)
        score = k_clf.score(X_test, y_test)
        print("Accuracy for user " + str(user_id) + " is " + str(k_clf.score(X_test, y_test)))

def lda_all():
    X_train, X_test, y_train, y_test = split_data()
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    print("Accuracy for all users is "+str(k_clf.score(X_test, y_test)))
    
def lda_all_sessions_separately():
    sessionIDs = np.unique(df.sessionID)
    for sessionID in sessionIDs:
        X_train, X_test, y_train, y_test = split_data_per_session(sessionID)
        k_clf = LinearDiscriminantAnalysis()
        k_clf.fit(X_train, y_train)
        score = k_clf.score(X_test, y_test)
        print("Accuracy for session " + str(sessionID) + " is " + str(score))

In [62]:
lda_all_users_separately()

Accuracy for user 1 is 1.0
Accuracy for user 2 is 1.0
Accuracy for user 3 is 1.0
Accuracy for user 4 is 1.0
Accuracy for user 5 is 1.0
Accuracy for user 6 is 1.0
Accuracy for user 7 is 1.0
Accuracy for user 8 is 1.0


In [65]:
lda_all()

Accuracy for all users is 0.8346883468834688


In [58]:
lda_all_sessions_separately()

Accuracy for session 1001 is 1.0
Accuracy for session 1002 is 1.0
Accuracy for session 1003 is 1.0
Accuracy for session 2001 is 1.0
Accuracy for session 2003 is 1.0
Accuracy for session 2004 is 1.0
Accuracy for session 2005 is 1.0
Accuracy for session 2006 is 1.0
Accuracy for session 2007 is 1.0
Accuracy for session 2008 is 1.0
Accuracy for session 2009 is 1.0
Accuracy for session 2010 is 1.0
Accuracy for session 2012 is 1.0
Accuracy for session 2013 is 1.0
Accuracy for session 2028 is 1.0
Accuracy for session 2029 is 1.0
Accuracy for session 2031 is 1.0
Accuracy for session 2032 is 1.0
Accuracy for session 3001 is 1.0
Accuracy for session 4001 is 1.0
Accuracy for session 4002 is 1.0
Accuracy for session 5001 is 1.0
Accuracy for session 6001 is 1.0
Accuracy for session 7001 is 1.0
Accuracy for session 7002 is 1.0
Accuracy for session 8002 is 1.0
Accuracy for session 8003 is 1.0
Accuracy for session 8010 is 1.0
Accuracy for session 8016 is 1.0
Accuracy for session 8017 is 1.0
Accuracy f